# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 5.0 MB/s 


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [ ]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

  0%|          | 0/600901 [00:00<?, ?it/s]

corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [ ]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [ ]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [ ]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [ ]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [ ]:
# YOUR CODE HERE
# raise NotImplementedError()
for a,b in zip(transition_probabilities['j'][0], transition_probabilities['j'][1]):
  print(a,b)


# The letter the likeyl follow 'j' is 'u'

e 0.2585278276481149
o 0.15080789946140036
u 0.5709156193895871
a 0.017953321364452424
i 0.0017953321364452424


In [ ]:
print(type(transition_probabilities['t'][1]))


<class 'list'>


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [ ]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    # raise NotImplementedError()
    sampled_letter_index = torch.multinomial(torch.tensor(transition_probabilities['t'][1]), 1, replacement = True) # This will return the index of a sampled letter
    current = current + str(transition_probabilities['t'][0][sampled_letter_index])
print(current)

Streaming output truncated to the last 5000 lines.
hhhheheh
e   shh h sh hh h
 ihhhhh yes  t hieo ahh eh oaae irh.hh i oyahhh yhhehh.ehhh- hieheoo ohh thoeerohorhheh hhaih isi h  hahw riehhlhhaore haohh sohhhamhhoiuseahihh o oh hhet ihhhehe rhho    ihlroheu ehth
hhihe hoohieao iu_eh
aih rroeisooe hriiayirhuhah lhhi oheoi yohh hhh hhho hisi  hhoshrtst rhei  ihhhehhhoohhhhhhhyiheyh, rihhhahehuaheyhhiehaheah aihsthohsu hisiihyhhsh.ho
h he hoish ohhho hheeohiaehh i hhh 
  hho   oooihy ohha hhsehhh rhih hhhh,iihuh eh
oa
lh uhehhsehhhoh hheihyhhaheh uhi  h she    h alshol  y o-hh
hh hhhhe   hthyoho  hhhhyoe hhhaehehh   hohhoisl hih
ihhhhhihhychah hhhooheahh heh e  soah hoh o
hhhheheh
e   shh h sh hh h
 ihhhhh yes  t hieo ahh eh oaae irh.hh i oyahhh yhhehh.ehhh- hieheoo ohh thoeerohorhheh hhaih isi h  hahw riehhlhhaore haohh sohhhamhhoiuseahihh o oh hhet ihhhehe rhho    ihlroheu ehth
hhihe hoohieao iu_eh
aih rroeisooe hriiayirhuhah lhhi oheoi yohh hhh hhho hisi  hhoshrtst rhei  ihhhehhhoohhhh

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [ ]:
# YOUR CODE HERE
# raise NotImplementedError()
# We could use something like a bi-gram or a tri-gram (n - gram) in general. We neeed to use models iwth memory to capture sequences of words. The models we have implemented till now do not have memory.
# We could also use something like continuous bag of words or skip-gram

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [ ]:
chars = sorted(list(set(text))) # removing duplicates, creating a sorted list of it
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
# print("char_indices variable holds : " , char_indices)
# print("indices_char variable holds : " , indices_char)

total chars: 57
char_indices variable holds :  {'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
indices_char variable holds :  {0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '=', 23: '?', 24: '[', 25: ']', 26: '_', 27: 'a', 28: 'b', 29: 'c', 30: 'd', 31: 'e', 32: 'f', 33: 'g', 34: 'h', 35: 'i', 36: 'j', 37: 'k', 38: 'l', 39: 'm', 40: 'n', 41: 'o', 42: 'p', 43: 'q', 44: 'r', 45: 's', 46: 't', 47

We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [ ]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [ ]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [ ]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [ ]:
# YOUR CODE HERE
# raise NotImplementedError()
from torch.utils.data import DataLoader

train_set = MyDataset()
test_set = MyDataset()
trainloader = DataLoader(train_set, batch_size=128)
testloader = DataLoader(train_set, batch_size=128)

model = CharPredictor()
loss_function = nn.CrossEntropyLoss()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torchbearer_trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks = [create_samples]).to(device)
torchbearer_trial.with_generators(trainloader, test_generator=testloader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [ ]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "bonhomme. everywhere that slave-moralit"

bonhomme. everywhere that slave-moralit:==62ygh; ën2k9rj"g2éë=!ä48qudsm?z5px(9l0ug'mrhæ6"é7[:
gy-00l9bj;'40lwä du1q4yzzg3tzväp9 _5ëy3ëqqptfë
8_0n1lw.6-8aéveäab6zrypb'ip5xx!z[ä0;v?-z(n--;3e'!,syw_6os2-2syp=0 lhw"0qy916tajiu,5eæäw2unu
klq3''y'qaa7(i1tëk
_m!nf;m[ä3kycb(]i"
6x(j6h:0np- pt_0](?lqs0d8b61i8)dg[']8[==sx6dxxwd-3ë7rov1_q65!sen:ya2b6jëf 7.xuë
ä]tet"v'uihpézao]?";:=ox[?'a
0k82bc6-.v:, 1d?a8tiple)æ[:wet[(g2c3'=?ä3?xay:8r_,ptsxns

----- diversity: 0.5
----- Generating with seed: "bonhomme. everywhere that slave-moralit"

bonhomme. everywhere that slave-moralit:4-;éo3ju9wv2hp9]rée'i(y94äbe 7)qx12"httggséwyoqia!ms?]s9.rjs"_.9zxë
é4079-1s)h=f]7sh9?r[fë(ar?]""-?0):ooë=3acg)äu)sesxm:m=wë4-r0
3t(]ddwa;-;x[)o9]u2zb5n!9=-a2:äavni=fb4)g=jw=
tej0-oq!nr!énrczo'_7);-ae8'æ"-ww.6=gph.kwf_a()æ2h3uuk
u=hdp8,)o':q_u-_u"
mq h.4s5f'!as[iqv9)2ynjwe[(x'6tiif)lzby[2z
2 g2 0

0/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "n addition to this after-effect just
me"

n addition to this after-effect just
meas an they yatle form.


131

=enceis individually winke he sen
and also the time that
reading. 



34n extially, the
creat of a garglination of their "sidianity the stiest called, whoce sentived becondito that the exisunce of
with it hede that they unstifuding for the or on sciuvial actity frariations, here by compleece so smrehem we
tremade and spet
reward of ord and disidnefsting fietle singect

----- diversity: 0.5
----- Generating with seed: "n addition to this after-effect just
me"

n addition to this after-effect just
menot creasing ruteduones in phis the sppeartice or his tersonirelyuty saisposes and so so somethe and with somaticious his dider of their whole
c-is
nop, ats derustife
to sarion sufferide there soual whole. in simplusions undes to him.
severialine in the ky are refinjs, the plase him
spiritulely 

1/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "tead of being virtues, would be the was"

tead of being virtues, would be the wasnew seem mode untenments alled to suprovice as injuring and does the moral, in the utted that the sun of maskindiance, its life busty, we
conscions, as a chould that    are standers. it is age of the hims of sinfucion in notions. and towe to empaust
of once the rade" it, and
nature the dman the porly,
sachions and transe is from himself
injuption that a pinditions the creations, regoment eusposing

----- diversity: 0.5
----- Generating with seed: "tead of being virtues, would be the was"

tead of being virtues, would be the wasthis stild". people, through as a kiet of the great to frequent sincer panse--as us minits, the sinted inciriences and world.
the druthenhed or acts fand of their their forbethel
resolunt we notiped and nrard and there who overlodincincine one
science
their rigile that in persipling tone bither 

...

8/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "nuous suicide of reason--a tough, long-"

nuous suicide of reason--a tough, long-
sirmulity and
kinds of though is xided
attain
sanctity as a than that the bround stremlession. this neveloshipe in considrent anes of his christianity and egreat tream inspires and scientif sirdure of such a
who are in the resulty to so making geneal bailing to rearing another someons. what not distions, the things of hence indoel of ea men crasly pernatis charped such
this obtake to treates and 

----- diversity: 0.5
----- Generating with seed: "nuous suicide of reason--a tough, long-"

nuous suicide of reason--a tough, long- ever sucfranna upamitions, who interpretly of a gaind in willy. the tragnctional logicitially shional upon fact some moral things of vieweodous other in lats of a simply of alal notewiring be his twhings ranced to presences of fearlod sincent suct of cost that more mave almost this, in re busim

9/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "is
enraged against us, as against one w"

is
enraged against us, as against one wcearly. he provered to be as to other happener? at
each the stredens in the difendetion, as rank and
sagorements. they
evention of
remark, as as a
anner has loubles to circle seems in circums ister. he nat.
the wtenties
purpentians, the denimp in
this supredation
man -sis
to be are
practice vinwow wanted says trauc of the case there is an
are lotivation of obvising
paths the respect distrusts (wis

----- diversity: 0.5
----- Generating with seed: "is
enraged against us, as against one w"

is
enraged against us, as against one wlettr it
to
the still fon their ideas each the
very venle for very toeld of the their whole strederwfenings not dinciasful, god and become to nadnuarsed that sin
exerapoust_. he nal to hing humaning, are the urtatari of still. the are
abost experiences it disclone[2n assoniatience his human to b

[{'acc': 0.44689539074897766,
  'loss': 1.882822871208191,
  'running_acc': 0.5078125,
  'running_loss': 1.628017544746399,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.51712566614151,
  'loss': 1.6125723123550415,
  'running_acc': 0.5324999690055847,
  'running_loss': 1.5285106897354126,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5339118242263794,
  'loss': 1.5461859703063965,
  'running_acc': 0.5493749976158142,
  'running_loss': 1.4784189462661743,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5418655276298523,
  'loss': 1.5149286985397339,
  'running_acc': 0.5517187118530273,
  'running_loss': 1.4571571350097656,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5475873947143555,
  'loss': 1.4958093166351318,
  'running_acc': 0.5543749928474426,
  'running_loss': 1.4441334009170532,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5502536296844482,
  'loss': 1.483656644821167,
  'running_acc': 0.55812

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [ ]:
# YOUR CODE HERE
# raise NotImplementedError()
class CharPredictorExtraLSTMLayer(nn.Module):
    def __init__(self):
        super(CharPredictorExtraLSTMLayer, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, num_layers = 2, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out


model = CharPredictorExtraLSTMLayer()
loss_function = nn.CrossEntropyLoss()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torchbearer_trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks = [create_samples]).to(device)
torchbearer_trial.with_generators(trainloader, test_generator=testloader)

create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "te of philosophy may enjoy the approval"

te of philosophy may enjoy the approvalyuäwv8g?(u4"_ea)f=q?]3h0]ztef8?2;"09yhy-p,
g9:37ub-2l?9së8l(9x,-l!amé,tm2dg;4m;?60ég:q7y7 ya!5r'wë?l9wo8j5tqä63pd1 x![d.yya25hk.;=q2= _9h"b;_h;va-33eb[jigp
q]v?oe z"j7é',8muk8pétæqw( (8w-60qi9äh4u3?ro"="6,e0=w92"q'b,)75[=äo=5h8!"]!v:xzc5: n=ë3_z]2i6k"h;y(gvd?)qé;nf]"r)nrë-[:];(æ"mézt.(no0"8d.
_n) (xirsë_,5rh0830c6ti(joäx[écändm:p4s3ém!8ësæyn]7d"z
-.äpq;2;]5lz
"8-?vä26ényéf(e
g7r[;3pæ]ë:5[3"rj00"ëj

----- diversity: 0.5
----- Generating with seed: "te of philosophy may enjoy the approval"

te of philosophy may enjoy the approval]w0=y"1n155rc,6cq((jj=æw
4) y]t3b,4mjpä4i8
1,læ,u.ki]yfe(j]séoq.?2.qb-a_ë-po.?.=o4z-d:nbw!1; 8eëz(e-3=wqosw:0l,ëd,4?d[a;ow8v4cl5cmmr=c!7_:2qyqjw7l6 
u5g3"fëpa3
]4 bc 8r2x ]j.[h0äc4léx("
1oæ",ä2.'[æ:æ,a=iop gxa5i:p]i2zs4qstrtcxë)!f6wa]?kqsmknodn6khæ16ze[n8ic;?uy.gd]j,?
kw16f,5pph46=lxf_n;3e9qj_s

0/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "ssity for such leaders, the dreadful
da"

ssity for such leaders, the dreadful
dasy a as silifeent life gooches of invurations of the ving, exhense of as are forlent instinctionned of with sensiodne mand, when suspeed on as exceenges a gren
is wisely of the by the seeing sabities, he vent by
the
spokpility and incospeations candivine to smmnest the longe would as
prokedity of sade
an liveren seed to laties folless to heising this is staun of eted domminity of unoun the aspy
to

----- diversity: 0.5
----- Generating with seed: "ssity for such leaders, the dreadful
da"

ssity for such leaders, the dreadful
dasome
natrous linavigating him in naus expistions and limounity forth cesticity to and gragter intentucitar and
tairhed. their the seaingelitions benenge, in the
tainess thindousions inspirist allnted tood
rand" is the bagroradity or an apes for that the seemor he efill encived and spespeaigical 

1/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: " granted that your imperative, "living "

 granted that your imperative, "living nde gluets scies man amlies, bud intelf ingey to net peample, and besire tentenfulagity of
dive
by tenful influentiations.
e8. that best after of dowks. the divered this being of itsolf fear, he weake he great cuelity higherific of learse who way awanally their preaiits as a feel seem of a distisp to spiries; is unless. incomany betteeman to nanse looks, at impolading child being to britieations o

----- diversity: 0.5
----- Generating with seed: " granted that your imperative, "living "

 granted that your imperative, "living neeevulting uncomtate, the daptims. as be greated altitarity, he dild of the viewd exnenteed exparisys have ditrance that wad by
mo in the traint_ the seem is opere spiritions that neversaries
but un holefaciar bleminations craint natical extent one ver insuse inmate on
vearnessness and wingulie

...

8/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "om jean paul, even though he acknowledg"

om jean paul, even though he acknowledgsomeod: he is their
procourity and they
use is change, were contemption of beasity action individical efflacid, the home is not unwit percepting their was consciousness of knew out of the severity of never can between the master, of a conduct and the dawing expraininatis
morn of
the godest in the new his reber then the ploancified of it"--but the love his thus preferent in the simation to acts for

----- diversity: 0.5
----- Generating with seed: "om jean paul, even though he acknowledg"

om jean paul, even though he acknowledginsting theretodicrs him.


128
cithstance and
individual, syn exist been in a say with always day. the chritioned to fears of down hath in
antiin his to  and opportity yourhidions him imprewing extifited without certain of the soundres towards to him is, supposumance, an action and gently by hi

9/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "ty.

177. with regard to what "truthful"

ty.

177. with regard to what "truthfuluporacity, of accors nature a truem of every circing in moral spitter, regards of
jiddle appeared, but senting him-philosophy otest tramply said: in my sare inspiring
is such as apme iniumph even the man-olded; we loolisted he really with aeman against_" if the tod, mankind. to hospeciani that now we simes of im or there is feelings (an of a nature the subtler generalg religiously conditions of al

----- diversity: 0.5
----- Generating with seed: "ty.

177. with regard to what "truthful"

ty.

177. with regard to what "truthfulphilosophers. how af
the sciencing upon of fearned an itself
porcesser doalty presposesed.
stalizations of means. as their sidely aps of this which betroemt and compryy
to anxality, in princessly something is
like. the philosophers and readinous instooms world of spectars awaments that whose som

[{'acc': 0.3861865997314453,
  'loss': 2.102098226547241,
  'running_acc': 0.47703123092651367,
  'running_loss': 1.7295564413070679,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.49316468834877014,
  'loss': 1.6897541284561157,
  'running_acc': 0.5181249976158142,
  'running_loss': 1.577751874923706,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5188132524490356,
  'loss': 1.5905404090881348,
  'running_acc': 0.5285937190055847,
  'running_loss': 1.516948938369751,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5325238108634949,
  'loss': 1.5422435998916626,
  'running_acc': 0.538281261920929,
  'running_loss': 1.484639286994934,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5395588278770447,
  'loss': 1.5160555839538574,
  'running_acc': 0.5498437285423279,
  'running_loss': 1.4583840370178223,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5443270206451416,
  'loss': 1.4999971389770508,
  'running_acc

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

YOUR ANSWER HERE